In [1]:
import scipy.sparse
import numpy as np

In [2]:
import pickle
import os

## Load data from MD analysis

Before building our SynD model, we used some Trp-cage MD data to build a Markov model. That analysis produced:
- A 10,500 state transition matrix
- A mapping of each state to a representative atomistic structure

### Load transition matrix

This 10,500-state transition matrix is our Markov model.

In [3]:
transition_matrix = scipy.sparse.load_npz('input/sparse_tmatrix.npz')

In [4]:
transition_matrix

<10500x10500 sparse matrix of type '<class 'numpy.float64'>'
	with 838020 stored elements in Compressed Sparse Row format>

## Load full-coordinate structure mapping

The "structure map" is simply a dictionary mapping each discrete Markov state to a full-coordinate atomistic structure.

This data is for Trp-cage, which has 272 atoms.

In [5]:
with open('input/coord_map.pkl', 'rb') as infile:
    full_coord_map = pickle.load(infile)

len(full_coord_map)

10500

In [6]:
full_coord_map[0].shape

(272, 3)

## Build the SynD model

Now, we'll construct our SynD model, which uses the transition matrix for dynamics. This transition matrix was built at a lagtime of 1ns, so 

The model directly generates discrete trajectories of integer states. We'll "backmap" that to an atomistic MD-like trajectory by just mapping each discrete state to an atomistic structure.

In [7]:
from synd.models.discrete.markov import MarkovGenerator

In [8]:
model = MarkovGenerator(
    transition_matrix=transition_matrix,
    backmapper=full_coord_map.get
)

Now, we can generate trajectories.

We'll generate two trajectories -- one starting from the folded state (1871), and one from another random state.

In [9]:
discrete_trajectories = model.generate_trajectory(
    initial_states=np.array([1871, 1337]),
    n_steps=10
)

discrete_trajectories.shape, discrete_trajectories

((2, 10),
 array([[1871,  648, 1255, 1282,  727, 1404, 1204, 1295,  650,  958],
        [1337, 2323, 4912, 2156, 3747, 2156,  529,  573, 1855, 2956]]))

Now, we can convert those discrete trajectories to atomistic trajectories.

In [10]:
atomistic_trajectories = model.backmap(discrete_trajectories)
atomistic_trajectories.shape

(2, 10, 272, 3)

Voila! Two 10ns atomistic trajectories, saved at 1ns timesteps.

## Saving our model

Now we save our model for later use. Note that the model, and everything we need to generate as many trajectories as we like, is a self-contained 45MB file.

In [11]:
model.save('output/trp-cage.synd')

In [12]:
size = os.stat('output/trp-cage.synd').st_size * 1e-6
print(f"Saved model is {size:.1f} MB")

Saved model is 45.0 MB
